# Lua III
## Metatables (Objects, sort of) & Review

## Metatables 
- Metatables are one of the newest additions to Lua
 - Added in Lua 5.0
- Metatables themselves are ordinary tables
 - They gain their special status through how we attach them to tables
- Metatables can be used to represent user-defined types
- All data types have metatables
 - Only the metatables of tables can be set and modified


![A graphical depection of a table, with an arrow pointing to another table to signify it is a metatable](metatable_intro.png)

## Basic Metatable Use


In [1]:
Window = {}
Window.prototype = {x = 0, y = 0, width = 100, height = 100}
Window.mt = {}

function Window.new(o)
    setmetatable(o,Window.mt)
    return o
end

Window.mt.__index = Window.prototype

w = Window.new({x = 10, y= 20})
z = Window.new{height = 100, width = 50}

In [2]:
w.width

100

In [3]:
z.x

0

## What can we do with Metatables
- Set Default values
- Limited Operator Overloading
 - Can only overload a small set of built in functions
 - No concept of method signature
- Reuse a metatable for multiple tables, ie a class
- Fake inheritance

## Setting Default Values
- To set default values when querying, we use the __\_\_index__ metamethod
 - What this actually does is provide a fall back table whose values are accses when the main table returns now for a given key
- As tables can hold functions, this allows a way to do inheritance

![A graphical depection of a table, with an arrow pointing to another table to signify it is a metatable. The __index key of the metatable points to another table, which serve to provide default values](metatable_index.png)

![A graphical depection of a table, with an arrow pointing to another table to signify it is a metatable. The __index key of the metatable points to itself, to indicate it is providing default values](metatable_self_index.png)

## Default Values Example

In [4]:
Building = {lat = 39.2555, long = 76.7113}
function Building.print(self)
    print("I'm located at " .. self.lat .. ", " .. self.long)
end
Building.__index = Building
UMBC = {}
setmetatable(UMBC,Building)

UMBC.print(UMBC)
UMBC:print()

I'm located at 39.2555, 76.7113
I'm located at 39.2555, 76.7113

## Inheritance Example

In [5]:
House = {numberOfBathrooms = 35}
function House.printBaths(self)
    print("I have " .. self.numberOfBathrooms .. " bathrooms.")
end

setmetatable(House,Building)
T = {}
T.__index = House
whiteHouse = {lat = 38.8977, long = 77.0336}
setmetatable(whiteHouse,T)--{__index = House})

whiteHouse.print(whiteHouse)
whiteHouse.printBaths(whiteHouse)
whiteHouse:print()

I'm located at 38.8977, 77.0336
I have 35 bathrooms.
I'm located at 38.8977, 77.0336

## Controlling the Creation of New Table Elements
- Just like the **__index** metamethod is called when an unknown element is accessed, the **__newindex** metamethod is used to update tables
- Defining this will prevent default behavior from occur
    - Causes tables to behave in a read-only like fashion
    - Can be bypassed using the `rawset` function

In [6]:
T.__newindex = function(t,k,v)
    print("No Updates Allowed")
end
whiteHouse.numberOfBathrooms = 4
whiteHouse.address = "1600 Pennsylvania Avenue"
print(whiteHouse.numberOfBathrooms)
print(whiteHouse.address)

No Updates Allowed
No Updates Allowed
35
nil

In [7]:
T.__newindex = function(t,k,v)
    --EXTRA CODE HERE
    rawset(t,k,v)
end
whiteHouse.numberOfBathrooms = 4
whiteHouse.address = "1600 Pennsylvania Avenue"
whiteHouse:printBaths()

I have 4 bathrooms.

## Metamethods for Libraries
- A field of a metatable or metamethod does not have to be limited to what Lua expects
- Several libraries can take advantage of metamethods to change their behavior.
    - **__tostring**
    - **__metatable**

In [8]:
print(whiteHouse)
T.__tostring = function(var)
    return "I'm located at " .. var.lat .. 
    "," .. var.long
end
print(whiteHouse)

table: 0x1d3c960
I'm located at 38.8977,77.0336

In [12]:
-- The __metatable Method is used to control the behavoir of the 
-- setmetatable and getmetatable functions
print(getmetatable(whiteHouse))
T.__metatable = "T"
print(getmetatable(whiteHouse))
--setmetatable(whiteHouse,T)
T.__metatable = nil
setmetatable(whiteHouse,T)

T
T

## Operator Overloading
- To overload a given operator, set the corresponding _metamethod_
 - To overload +, define \_\_add (two underscores)
 - Each metatable only has one metamethod of each name
  - To account for differing operand types, a long sequence of __if__ statements may be needed

## Operator Overloading Example
- Suppose we have a metatable for the class of complex numbers

In [13]:
--Based off Fabio Mascarenhas' example
mt = {}
function mt.__add(c1,c2)
    if(getmetatable(c1) ~= mt) then
        res = {r = c1 + c2.r , i = c2.i}
        setmetatable(res,mt)
        return res
    end
    if(getmetatable(c2) ~= mt) then
        res = {r = c1.r + c2, i = c1.i}
        setmetatable(res,mt)
        return res
    end
    res = {r = c1.r + c2.r , i = c1.i + c2.i}
    setmetatable(res,mt)
    return res
end

In [14]:
complex = {r = 1, i = 2}
complex2 = {r = 2, i = 4}
setmetatable(complex,mt)
setmetatable(complex2,mt)
x = 5 + complex
--y = 5 - complex
y = complex + 5
z = complex + complex2
print(string.format("%d + %di",x.r,x.i))
print(string.format("%d + %di",y.r,y.i))
print(string.format("%d + %di",z.r,z.i))

6 + 2i
6 + 2i
3 + 6i

## Other Overloadable Operations
- **__mul**
- **__sub**
- **__div**
- **__unm**
- **__pow**
- **__eq**
- **__lt**
- **__le** 

## OOP in Lua
- We will discuss object oriented programming in detail over the next few classes
- In Lua, we can approximate an object by
 - Using metatables to define a class
 - Use the __:__ operator as a shorthand for passing the current value to a function
 

In [15]:
Animal = {species = 'Dog', noise = "woof"}

function Animal.makeNoise(animal)
    print(animal.species .. " goes " .. animal.noise)
end

Animal.makeNoise(Animal)


Dog goes woof

In [16]:
Animal2 = {species = "Dog" , noise = "woof"}

--function Animal2.makeNoise(self,punct)
function Animal2:makeNoise(punct)
    print( self.species .. " goes " .. self.noise .. punct )
end

Animal2:makeNoise("!")
Animal2.makeNoise(Animal2,"!")

Dog goes woof!
Dog goes woof!

In [17]:
Animal3 = {species = "Dog" , noise = "woof"}

function Animal3.makeNoise(self)
    print( self.species .. " goes " .. self.noise )
end

Animal3:makeNoise()

Dog goes woof

## Object Constructors
- In Lua, the constructor becomes a function that
    - creates a table
    - sets the appropriate metatable
    - and returns the object

In [18]:
function Animal:new( m_species, m_noise)
    o = {species = m_species, noise = m_noise}
    setmetatable(o, self)
    self.__index = self
    return o
end

function Animal:makeNoise()
    print( self.species .. " goes " .. self.noise )
end

In [24]:
dog = Animal:new("Dog","Woof")
dog:makeNoise()

cat = Animal:new("Cat","Meow")
cat:makeNoise()

llama = Animal.new(Animal, "Llama","moo")
llama.makeNoise(llama)

Dog goes Woof
Cat goes Meow
Llama goes moo

## Issues with OOP in Lua
- Can be cumbersome
- An object is still just a table
- Metatables can be changed
- Consider the output of the following:

In [20]:
type(Animal3)

table

In [21]:
Window = {}
Window.prototype = {x = 0, y = 0, width = 100, height = 100}
Window.mt = {}
Window.mt2 = {}

function Window.new(o)
    setmetatable(o,Window.mt)
    return o
end

Window.mt.__index = Window.prototype
Window.mt2.__index = {color = "red", shape = "circle"}

In [22]:
x = Window.new{height = 200}
print(x.x,x.y,x.width,x.height)
setmetatable(x,Window.mt2)
print(x.x,x.y,x.width,x.height)

x = {10, 20, 30, 40}
print(x[1])

0	0	100	200
nil	nil	nil	200
10

## Lua Wrap Up and Analysis
- To finish up our discussion of Lua, we are going to run through some things from chapters 5-10 of the textbook

## Names, Binding, and Scope
- Names
 - Case sensitivity?
 - Can reserved words be used?
 - What characters are allowed?
- Binding 
 - When does type binding occur?
- Scope
 - What is the default scope of a variable? 

## Data Types
- What are the data types?
- Can the user define their own data types?
- Arrays 
 - Are subscripts checked?
 - Can we use slicing?
 - How well supported are multidimensional arrays?
- Are pointers accessible?
- Is there type checking?

## Expressions and Assignments
- Does Lua have operator precedence?
- Does Lua allow operator overloading?
- How are type conversions done?
- Does Lua have compound assignment operators (ie +=) ?

## Control Structures
- What is the syntax of an if statement?
- Does Lua have a multiple-select structure (ie switch)?
- What are Lua's counter controlled loops
- What are Lua's logic controlled loops?

## Subprograms
- What is the function syntax in Lua
- Does Lua allow functions to be passed as parameters of other functions?
- Are functions in Lua type-checked
- Can functions in Lua return more than one value?
- Can function definitions be nested?
- Does Lua allow closures?


## Lua Example Application
- A common task in natural language processing is to be able to read a file and calculate various statistics on the words in that file
- As an in-class exercise, we will write together a program that does the following
 - Reads in a text file and breaks it into words, based on spacing.
 - Counts the frequency of each word
 - Prints a the most common words in the file


## Lua Example Application 
### Step 1: Read in the file 

In [36]:
file = io.open("words.txt")
text = file:read('*all')
words = {}

for word in text:gmatch("%w+") do
    table.insert(words,word)
end    


## Lua Example Application 
### Step 2: Count the words

In [39]:
counts = {}
for _,word in pairs(words) do
    if counts[word] then
        counts[word] = counts[word] + 1
    else
        counts[word] = 1
    end
end


## Lua Example Application 
### Step 3: Print Most Common

In [44]:
counts_array = {}

for word, count in pairs(counts) do
        table.insert(counts_array, {w = word, c = count})
end
table.sort(counts_array,function(a,b) return a.c > b.c end)

for _, together in pairs(counts_array) do
    print(together.w .. ": " .. together.c)
end

of: 7
to: 7
you: 5
the: 5
know: 5
I: 5
in: 4
a: 4
is: 4
which: 3
this: 3
it: 3
by: 3
are: 3
matter: 2
Holmes: 2
You: 2
Mr: 2
we: 2
him: 2
have: 2
can: 2
that: 2
But: 2
Sherlock: 2
and: 2
me: 2
his: 1
sir: 1
hardly: 1
In: 1
Henry: 1
nothing: 1
cosy: 1
hailed: 1
four: 1
My: 1
Windigate: 1
quivering: 1
business: 1
everything: 1
Brixton: 1
named: 1
name: 1
who: 1
Road: 1
member: 1
could: 1
Baker: 1
Oakshott: 1
don: 1
geese: 1
Oh: 1
very: 1
turn: 1
anything: 1
Mrs: 1
was: 1
farther: 1
assisting: 1
fellow: 1
trace: 1
endeavouring: 1
fingers: 1
meet: 1
go: 1
with: 1
sold: 1
pleasure: 1
market: 1
before: 1
wheeler: 1
people: 1
cried: 1
club: 1
were: 1
better: 1
longed: 1
Breckinridge: 1
little: 1
explain: 1
whom: 1
what: 1
t: 1
how: 1
outstretched: 1
Alpha: 1
passing: 1
tell: 1
he: 1
other: 1
wind: 1
some: 1
Excuse: 1
interested: 1
salesman: 1
case: 1
am: 1
my: 1
pray: 1
hands: 1
Who: 1
room: 1
place: 1
discuss: 1
said: 1
rather: 1
man: 1
How: 1
swept: 1
than: 1
had: 1
It: 1